In [1]:
import numpy as np
import os
import cv2
import time
from tqdm.auto import tqdm
import torch.nn.functional as F

c:\Users\jasso\.conda\envs\computer-vision\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
names = ['1 RIBU ASLI', '1 RIBU PALSU', '10 RIBU ASLI', '10 RIBU PALSU', 
         '100 RIBU ASLI', '100 RIBU PALSU', '20 RIBU ASLI', '20 RIBU PALSU', 
         '5 RIBU ASLI', '5 RIBU PALSU', '50 RIBU ASLI', '50 RIBU PALSU']

In [3]:
dataset_path = "C:\Codes\Computer Vision\Project\PROJECT-DATA-UANG-11"

In [4]:
def load_dataset_with_folder_structure(folder_path,augment=True):
    images_folder = os.path.join(folder_path, "images")
    labels_folder = os.path.join(folder_path, "labels")
    
    data = []
    labels = []
    count=0
    
    for img_name in os.listdir(images_folder):
        if img_name.endswith(".jpg"): 
            img_path = os.path.join(images_folder, img_name)
            label_path = os.path.join(labels_folder, os.path.splitext(img_name)[0] + ".txt")  # File label

            img = cv2.imread(img_path)
            if img is not None and os.path.exists(label_path):  # Pastikan gambar dan label ada
                resized_img = cv2.resize(img, (512, 512))  # Resize gambar
                data.append(resized_img)
                
                with open(label_path, "r") as f:
                    first_line = f.readline().strip()  # Ambil baris pertama
                    class_index = int(first_line.split()[0])  # Digit pertama sebagai label
                    labels.append(class_index)
    
    
    return np.asarray(data, dtype=np.uint8), np.array(labels, dtype=np.int32)

In [5]:
x_val, y_val=load_dataset_with_folder_structure(dataset_path+'/valid/', False)
x_test, y_test=load_dataset_with_folder_structure(dataset_path+'/test/', False)
x_train, y_train=load_dataset_with_folder_structure(dataset_path+'/train/', False)

In [6]:
import torch
from torchvision import models

In [7]:
class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = torch.nn.Linear(in_features=2048, out_features=12)
        # self.resnet.compile()
        
    def forward(self, x):
        return self.resnet(x)

In [8]:
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [9]:
class MoneyDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        image = self.x[idx]
        label = self.y[idx]

        if self.transform:
            image = self.transform(image)
            label = torch.tensor(label)

        return image, label

In [10]:
transform = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = MoneyDataset(x_train, y_train, transform)
val_dataset = MoneyDataset(x_val, y_val, transform)
test_dataset = MoneyDataset(x_test, y_test, transform)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [12]:
test_loader = DataLoader(test_dataset)

In [13]:
len(train_loader)

320

In [14]:

def run_one_epoch(
    epoch: int,
    ds_sizes,
    dataloaders,
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    loss: nn.Module,
    scheduler: torch.optim.lr_scheduler,
    criterion: nn.Module
):
    global best_AUROC
    
    metrics = {}
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    for phase in ["train", "val"]:

        if phase == "train":
            model.train()
        else:
            model.eval()
        
        avg_loss = 0

        for images, labels in tqdm(dataloaders[phase], total=len(dataloaders[phase])):
            labels = labels.type(torch.LongTensor)
            images = images.to(device)
            labels = labels.to(device)
            labels_onehot = F.one_hot(labels, num_classes=12).float()

            # Zero the gradients
            optimizer.zero_grad()

            # Track history if in phase == "train"
            with torch.set_grad_enabled(phase == "train"):
                outputs = model(images)
                
                # print(outputs)
                # print(labels_onehot)
                loss = criterion(outputs, labels_onehot)
                
                if phase == "train":
                    loss.backward()
                    optimizer.step()

            avg_loss += loss.item() * images.size(0)

        epoch_loss = avg_loss / ds_sizes[phase]
        
        # step the scheduler
        if phase == "train":
            scheduler.step(epoch_loss)
        

        # Metrics tracking
        if phase == "train":
            metrics["train_loss"] = round(epoch_loss, 3)
        else:
            metrics["val_loss"] = round(epoch_loss, 3)

    return metrics

In [15]:
def train(dataloaders, ds_sizes, model, optimizer, criterion, scheduler):
    for epoch in range(20):
        start = time.time()

        metrics = run_one_epoch(
            epoch=epoch,
            ds_sizes=ds_sizes,
            dataloaders=dataloaders,
            model=model,
            optimizer=optimizer,
            loss=criterion,
            scheduler=scheduler,
            criterion=criterion
        )

        end = time.time() - start

        print(f"Epoch {epoch+1} completed in: {round(end/60, 3)} mins, val_loss: {metrics['val_loss']}, train_loss: {metrics['train_loss']}")
    return model

In [16]:
model = ResNet50().to('cuda')
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

dataloaders = {
    "train": train_loader,
    "val": val_loader,
    "test": test_loader
}

ds_sizes = {
    "train": len(train_dataset),
    "val": len(val_dataset),
    "test": len(test_dataset)
}


train(dataloaders, ds_sizes, model, optimizer, criterion, scheduler)

c:\Users\jasso\.conda\envs\computer-vision\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jasso\.conda\envs\computer-vision\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 69/69 [00:06<00:00, 11.30it/s]


Epoch 1 completed in: 2.334 mins, val_loss: 0.699, train_loss: 0.425


100%|██████████| 69/69 [00:05<00:00, 11.60it/s]


Epoch 2 completed in: 1.996 mins, val_loss: 0.953, train_loss: 0.093


100%|██████████| 69/69 [00:05<00:00, 12.49it/s]


...

100%|██████████| 69/69 [00:05<00:00, 11.85it/s]


Epoch 19 completed in: 1.953 mins, val_loss: 0.031, train_loss: 0.012


100%|██████████| 69/69 [00:05<00:00, 12.90it/s]

Epoch 20 completed in: 1.953 mins, val_loss: 0.03, train_loss: 0.009


ResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [17]:
model.eval()

ResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [28]:
#predict
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.type(torch.LongTensor)
        images = images.to('cuda')
        labels = labels.to('cuda')
        print(images.shape)
        labels_onehot = F.one_hot(labels, num_classes=12).float()
        print(labels_onehot)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        print(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f"predicted: {predicted}, actual: {labels}")

print(correct)
print(total)
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

torch.Size([1, 3, 512, 512])
tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[ -0.8181,  -5.7254,  -1.1015, -13.6726,  10.9881, -21.7550,  -4.4642,
         -18.0331,  -4.8847,  -6.7337,  -2.0833, -29.8839]], device='cuda:0')
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0')
torch.Size([1, 3, 512, 512])
tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[ -0.9238,  -5.9773,  -1.1505, -13.8654,  10.8354, -21.4754,  -4.3825,
         -17.8293,  -4.7108,  -6.5030,  -2.1256, -29.7291]], device='cuda:0')
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0')
torch.Size([1, 3, 512, 512])
tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[ -0.9974,  -5.7537,  -1.3097, -13.4500,  10.5989, -20.9154,  -4.4501,
         -17.3912,  -4.7488,  -6.7706,  -1.5319, -28.7884]], device='cuda:0')
predicted: tensor([4], device='cuda:0'), actual: tensor([4

KeyboardInterrupt: 

In [19]:
torch.save(model.state_dict(), 'model.pth')

In [29]:
model = ResNet50()
model.load_state_dict(torch.load('resnet50-3.pth'))
model.to('cuda')
model.eval()

c:\Users\jasso\.conda\envs\computer-vision\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jasso\.conda\envs\computer-vision\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\jasso\AppData\Local\Temp\ipykernel_42092\664463377.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.c

ResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [32]:
#predict
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to('cuda')
        labels = labels.to('cuda')
        labels = labels.type(torch.LongTensor)
        labels = F.one_hot(labels, num_classes=12).float()

        outputs = model(images)
        print(outputs)
        predicted = torch.argmax(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels)).sum().item()
        print(f"predicted: {predicted}, actual: {torch.argmax(labels)}")

print(correct)
print(total)
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

tensor([[ -2.6116,  -7.1317,  -2.3266, -11.7431,  13.5664, -12.2689,  -1.2639,
         -12.4482,  -2.4917,  -8.1012,  -2.3458, -22.5334]], device='cuda:0')
predicted: tensor([4], device='cuda:0'), actual: 4
tensor([[ -2.5884,  -7.0823,  -2.3517, -11.9226,  13.4363, -11.7013,  -0.9943,
         -12.0384,  -2.6185,  -8.1260,  -2.2005, -22.3511]], device='cuda:0')
predicted: tensor([4], device='cuda:0'), actual: 4
tensor([[ -2.5843,  -7.2236,  -2.5374, -11.2960,  13.0993, -12.1865,  -1.5803,
         -12.5869,  -2.3771,  -7.7384,  -1.8997, -22.0820]], device='cuda:0')
predicted: tensor([4], device='cuda:0'), actual: 4
tensor([[ -2.5647,  -7.2062,  -2.4941, -11.3428,  13.1717, -12.4404,  -1.7128,
         -12.7787,  -2.3255,  -7.7879,  -2.0820, -22.2176]], device='cuda:0')
predicted: tensor([4], device='cuda:0'), actual: 4
tensor([[ -2.5454,  -6.9391,  -2.6281, -11.4630,  13.0182, -12.1347,  -1.7705,
         -12.5726,  -2.5146,  -7.8072,  -2.2037, -21.9767]], device='cuda:0')
predicted: 

KeyboardInterrupt: 